1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты. 
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, precision_recall_curve, confusion_matrix

In [2]:
df = pd.read_csv("churn_data.csv")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [4]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [5]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [6]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))

for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [7]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Создадим функцию для расчёта выручки. (Не уверен, правильно ли понял как её считать)

In [8]:
def calc_profit(cnf_matrix, simple_output=False):
    
    tp = cnf_matrix[0][0]
    fp = cnf_matrix[0][1]
    fn = cnf_matrix[1][0]
    tn = cnf_matrix[1][1]
    
    profit = tp * 2 - fp * 1
    
    if simple_output:
        return profit
    else:
        print(f'Ожидаемая прибыль текущей модели = ${profit}')

### RandomForest

In [9]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth=None, max_features=0.5, 
                                          min_samples_leaf=3, random_state=42)),
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary')),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('classifier',
                 RandomForestClassifier(max_features=0.5, min_samples_leaf=3,
                                        random_state=42))])

In [10]:
preds = pipeline.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])

metrics_list = []
metrics_list.append([precision[ix], recall[ix], fscore[ix], calc_profit(cnf_matrix, simple_output=True)])
pd.DataFrame(metrics_list[-1], index = ['precision', 'recall', 'f1score', 'profit'])

,0
precision,0.661323
recall,0.648330
f1score,0.654762
profit,3475.000000


#### Вопрос 1: объясните своими словами смысл метрик Precison, Recall *
1. Какова их взаимосвязь и как с ними связан порог вероятности? 
2. Можно ли подобрать порог так, что recall будет равен 1? Что при этом будет с precision
3. Аналогичный вопрос про precision

#### Ответ:

Высокий Precision минимизирует ошибки ложного срабатывания (False Positive), в то время как высокий Recall - ошибки ложного пропуска (False Negative). На какую из метрик мы будем обращать больше внимания для оценки качества классификации зависит от поставленной задачи.

1. Зачастую, меняя порог вероятности, мы можем сами выбирать на какую метрику нужно сделать упор. Дело в том, что чем больше будет ошибок ложного срабатывания, тем меньше ошибок ложного пропуска. И наоборот.

2. Можно подобрать порог вероятности так, чтобы Recall был равен 1, однако скорее всего Precision будет намного ниже.

3. Precision так же можно сделать равным 1, но Recall в таком случаем будет ниже.

Для нахождения оптимального порога часто используют F-меру, которая является гармоническим средним между Precision и Recall.

<b>Вопрос 2: предположим, что на удержание одного пользователя у нас уйдет 1 доллар. При этом средняя ожидаемая прибыль с каждого TP (true positive) - 2 доллара. Оцените качество модели выше с учетом этих данных и ответьте на вопрос, является ли она потенциально экономически целесообразной?</b>

Ваш ответ здесь:  

Не уверен, правильно ли понял, как считать доходность модели, подсказка сбивает с толку. Как мне видится, если мы должны получать по $2 за каждого удержанного, то это будет TrueNegative, а не TruePositive.
В таком случае посчитать доходность можно будет по формуле 2TN - 1TN - 1FP.

Однако судя по тому, что есть, модель можно считать экономически целесообразной.

### GradientBoosting

In [11]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier())
])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Balance')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary')),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('classifier', GradientBoostingClassifier())])

In [13]:
preds = pipeline.predict_proba(X_test)[:, 1]

In [14]:
precision, recall, tresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)

ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])

metrics_list.append([precision[ix], recall[ix], fscore[ix], calc_profit(cnf_matrix, simple_output=True)])
pd.DataFrame(metrics_list[-1], index = ['precision', 'recall', 'f1score', 'profit'])

,0
precision,0.703704
recall,0.597250
f1score,0.646121
profit,3715.000000


### LogisticRegression

In [15]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression())
])

In [16]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Balance')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary')),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('classifier', LogisticRegression())])

In [17]:
preds = pipeline.predict_proba(X_test)[:, 1]

In [18]:
precision, recall, tresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)

ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])

metrics_list.append([precision[ix], recall[ix], fscore[ix], calc_profit(cnf_matrix, simple_output=True)])
pd.DataFrame(metrics_list[-1], index = ['precision', 'recall', 'f1score', 'profit'])

,0
precision,0.46240
recall,0.56778
f1score,0.50970
profit,3619.00000


Судя по метрикам, наилучшая модель - GradientBoosting, поскольку она обеспечивает лучший показатель Precision.

Мы расчитываем прибыль по формуле $profit = TP \cdot 2 - FP \cdot 1$, это значит, что для увиличения прибыли нужно как можно больше меток True Positive и как можно меньше False Positive, а именно это нам и дает максимизация Precision.

In [19]:
pd.DataFrame(metrics_list, 
            index=['RandomForest', 'GradientBoosting', 'LogisticRegression'],
            columns=['Precision', 'Recall', 'F1score', 'Profit'])

,Precision,Recall,F1score,Profit
RandomForest,0.661323,0.64833,0.654762,3475
GradientBoosting,0.703704,0.59725,0.646121,3715
LogisticRegression,0.462400,0.56778,0.509700,3619
